In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import altair as alt

In [2]:
df = pd.read_csv("globalterrorismdb_0718dist.csv",encoding = "ISO-8859-1",engine="python")

In [3]:
df.head()

,eventid,iyear,imonth,iday,approxdate,extended,resolution,country,country_txt,region,...,addnotes,scite1,scite2,scite3,dbsource,INT_LOG,INT_IDEO,INT_MISC,INT_ANY,related
0,197000000001,1970,7,2,NaN,0,NaN,58,Dominican Republic,2,...,NaN,NaN,NaN,NaN,PGIS,0,0,0,0,NaN
1,197000000002,1970,0,0,NaN,0,NaN,130,Mexico,1,...,NaN,NaN,NaN,NaN,PGIS,0,1,1,1,NaN
2,197001000001,1970,1,0,NaN,0,NaN,160,Philippines,5,...,NaN,NaN,NaN,NaN,PGIS,-9,-9,1,1,NaN
3,197001000002,1970,1,0,NaN,0,NaN,78,Greece,8,...,NaN,NaN,NaN,NaN,PGIS,-9,-9,1,1,NaN
4,197001000003,1970,1,0,NaN,0,NaN,101,Japan,4,...,NaN,NaN,NaN,NaN,PGIS,-9,-9,1,1,NaN


In [70]:
grouped = df.sample(n=20000)

In [71]:
grouped_df1 = grouped.groupby(['iyear', 'attacktype1_txt','region_txt','targtype1_txt']).size().reset_index(name='count')
grouped_df2 = grouped.groupby(['iyear','region_txt','targtype1_txt']).size().reset_index(name='count')

In [72]:
grouped_df1['Country'] = grouped.country_txt
grouped_df1['City'] = grouped.city
grouped_df1['Weapon_Type'] = grouped.weaptype1_txt
grouped_df1

,iyear,attacktype1_txt,region_txt,targtype1_txt,count,Country,City,Weapon_Type
0,1970,Armed Assault,Middle East & North Africa,Military,1,Dominican Republic,Santo Domingo,Unknown
1,1970,Armed Assault,Middle East & North Africa,Police,1,NaN,NaN,NaN
2,1970,Armed Assault,North America,Police,1,NaN,NaN,NaN
3,1970,Armed Assault,South America,Military,2,NaN,NaN,NaN
4,1970,Armed Assault,Western Europe,Government (Diplomatic),1,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
6171,2017,Unknown,Sub-Saharan Africa,Government (Diplomatic),1,NaN,NaN,NaN
6172,2017,Unknown,Sub-Saharan Africa,Military,13,NaN,NaN,NaN
6173,2017,Unknown,Sub-Saharan Africa,Police,1,NaN,NaN,NaN
6174,2017,Unknown,Sub-Saharan Africa,Private Citizens & Property,5,NaN,NaN,NaN


In [73]:
x_axis_ticks = grouped_df1['iyear'].unique().tolist()
x_ticks = x_axis_ticks[::2]
x_ticks

[1970,
 1972,
 1974,
 1976,
 1978,
 1980,
 1982,
 1984,
 1986,
 1988,
 1990,
 1992,
 1995,
 1997,
 1999,
 2001,
 2003,
 2005,
 2007,
 2009,
 2011,
 2013,
 2015,
 2017]

In [74]:
alt.data_transformers.disable_max_rows()
selection = alt.selection(type='interval')
selection2 = alt.selection_multi(fields=['attacktype1_txt'], bind='legend')
selection3 = alt.selection(type='interval')
selection4 = alt.selection_multi(fields=['region_txt'], bind='legend')
options = grouped_df1['targtype1_txt'].drop_duplicates().tolist()
new = []
new.append(None)
dropdown_options = new+options
dropdown = alt.binding_select(
    options=dropdown_options,
    name='Target Types',
    labels = ["ALL"]+dropdown_options[1:]
)

selection5 = alt.selection_single(
    fields=['targtype1_txt'],
    #value=[{'targtype1_txt': dropdown_options[0]}],
    bind=dropdown
)
years = alt.selection_interval(encodings=['x'])
bar_slider = alt.Chart(grouped_df1).mark_bar().encode(
    x='iyear',
    y=alt.Y('count()',axis = alt.Axis(title='# Attacks')),
    color = alt.value('red')).properties(height=50,width=800).add_selection(years)
# Create scatter plot
legend_attack = grouped_df1['attacktype1_txt'].drop_duplicates().tolist()
legend_region = grouped_df1['region_txt'].drop_duplicates().tolist()
scatter1 = alt.Chart(grouped_df1).mark_bar().encode(
    x=alt.X('iyear:O',axis=alt.Axis(values=x_ticks)),
    y = 'count:Q',
    #y=alt.Y('count:Q',scale=alt.Scale(domain=[0,2000])),
    #color=alt.Color('attacktype1_txt:N', legend=alt.Legend(orient='right')),
    color=alt.condition(selection2&selection3&selection4&selection5,
                        alt.Color('attacktype1_txt:N',
                                  legend=alt.Legend(orient='left',title='Attack Types',values=legend_attack)),
                        alt.value('lightgray'),
                       scale = alt.Scale(scheme = 'tableau10')),
    #shape = alt.condition(selection3,alt.Shape('attacktype1_txt:N'),alt.value('cross')),
    #shape = 'targtype1_txt:N',
    tooltip=['Country', 'City','Weapon_Type']
).add_selection(
    selection
).add_selection(
    selection2
).add_selection(
    selection5
).transform_filter(years).properties(width=400,height=200,title='Total Attacks By Attack-Type By Years')
scatter2 = alt.Chart(grouped_df1).mark_bar().encode(
    x=alt.X('iyear:O',axis=alt.Axis(values=x_ticks)),
    y=alt.Y('count:Q'),
    #color=alt.Color('region_txt:N', legend=alt.Legend(orient='right')),
    color=alt.condition(selection4&selection&selection2&selection5,
                        alt.Color('region_txt:N',
                                  legend=alt.Legend(orient='right',title='Regions',values=legend_region),
                                 scale = alt.Scale(scheme = 'tableau10')),
                        alt.value('lightgray')),
    #shape = alt.Shape('targtype1_txt:N',legend=None),
    #shape = 'targtype1_txt:N',
    tooltip=['Country', 'City','Weapon_Type']
).add_selection(
    selection3
).add_selection(
    selection4
).add_selection(
    selection5
).transform_filter(years).properties(width=400,height=200,title ='Attacks On Different Regions By Years')
charts = alt.hconcat(scatter1, scatter2).resolve_legend()
combined = charts&bar_slider
combined.save("visualisation3.html")
combined

C:\Users\user\anaconda3\lib\site-packages\altair\utils\deprecation.py:65: AltairDeprecationWarning: 'selection' is deprecated.
   Use 'selection_point()' or 'selection_interval()' instead; these functions also include more helpful docstrings.
  warnings.warn(message, AltairDeprecationWarning)
C:\Users\user\anaconda3\lib\site-packages\altair\utils\deprecation.py:65: AltairDeprecationWarning: 'selection_multi' is deprecated.  Use 'selection_point'
  warnings.warn(message, AltairDeprecationWarning)
C:\Users\user\anaconda3\lib\site-packages\altair\utils\deprecation.py:65: AltairDeprecationWarning: 'selection_single' is deprecated.  Use 'selection_point'
  warnings.warn(message, AltairDeprecationWarning)
C:\Users\user\anaconda3\lib\site-packages\altair\utils\deprecation.py:65: AltairDeprecationWarning: 'add_selection' is deprecated. Use 'add_params' instead.
  warnings.warn(message, AltairDeprecationWarning)


alt.VConcatChart(...)